This is based on our official notebook - not something I can take credit for :)
https://github.com/planetlabs/planet-amazon-deforestation/blob/master/planet_chip_examples.ipynb

# *Planet: Understanding the Amazon from Space* challenge

This notebook will show you how to do some basic manipulation of the images and label files.

In [2]:
import sys
import os
import subprocess

from six import string_types

# Make sure you have all of these packages installed, e.g. via pip
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy
from skimage import io
from scipy import ndimage
from IPython.display import display
%matplotlib inline

## Setup
Set `PLANET_KAGGLE_ROOT` to the proper directory where we've got the TIFF and JPEG zip files, and accompanying CSVs.

In [3]:
PLANET_KAGGLE_ROOT = os.path.abspath("../input/planets-dataset/planet/planet")
PLANET_KAGGLE_JPEG_DIR = os.path.join(PLANET_KAGGLE_ROOT, 'train-jpg')
PLANET_KAGGLE_LABEL_CSV = os.path.join(PLANET_KAGGLE_ROOT, 'train_classes.csv')
assert os.path.exists(PLANET_KAGGLE_ROOT)
assert os.path.exists(PLANET_KAGGLE_JPEG_DIR)
assert os.path.exists(PLANET_KAGGLE_LABEL_CSV)

## Inspect image labels
The labels are in a CSV entitled `train.csv`. Note that each image can be tagged with multiple tags. We'll convert them to a "one hot" style representation where each label is a column:

In [4]:
labels_df = pd.read_csv(PLANET_KAGGLE_LABEL_CSV)
labels_df.head()

In [5]:
# import dask.dataframe as dd
# labels_df = dd.read_csv(PLANET_KAGGLE_LABEL_CSV, blocksize=50e6)
# labels_df.head()

In [6]:
# Build list with unique labels
label_list = set()
for row in range(len(labels_df)):
    row_labels = labels_df['tags'][row].split(' ')
    label_list.update(row_labels)
    
label_list = list(label_list)
label_list

In [7]:
# Add onehot features for every label
for label in label_list:
    labels_df[label] = labels_df['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)
# Display head
labels_df.head()


In [8]:
# Histogram of label instances
labels_df[label_list].sum().sort_values().plot.bar()

In [9]:
for i in range(9):
    plt.subplot(330 + 1 + i)
    file = PLANET_KAGGLE_JPEG_DIR + '/' + 'train_' + str(i) + '.jpg'
    image = plt.imread( file)
    plt.imshow(image)

    

In [10]:
from keras.preprocessing.image import load_img, img_to_array
images = list()
for file in os.listdir(PLANET_KAGGLE_JPEG_DIR):
    image = load_img(PLANET_KAGGLE_JPEG_DIR + '/' + file, target_size=(128,128))
    image = img_to_array(image, dtype='uint8')
    images.append(image)

In [11]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.optimizers import Adam

In [12]:
X = np.asarray(images, dtype='float16')/255
y = np.asarray(labels_df[label_list], dtype='uint8')

print(X.shape, y.shape)

In [13]:

nRows,nCols,nDims = X.shape[1:]
X = X.reshape(X.shape[0], nRows, nCols, nDims)

X  /= 255
X.shape



In [14]:
import gc

gc.collect()

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state = 1)

In [16]:
del X 
del y
gc.collect

In [17]:
import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [18]:

model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)),
    Conv2D(32, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.1),

    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.1),

    Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.1),

    Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(256, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.1),

    Flatten(),

    Dense(1024, activation='relu'),
    Dense(17, activation='sigmoid') 
])

In [19]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
model_fit = model.fit(
    X_train, y_train,
    batch_size=128,
    epochs=5,
    verbose=1,
    validation_data=(X_val, y_val)
)

In [ ]:
y_pred = model.predict(X_val, batch_size=128)
score = fbeta_score(y_val, np.array(y_pred) > 0.2, beta=2, average='samples')

print("F beta score: ", score)
print("Error: %.2f%%" % (100 - score * 100))